# Programa principal

In [ ]:
import numpy as np
import random

def simul_main():
  #Inicializamos variables a 0.
  global T,t,TAterrizaje,TDespegue,ColaAterrizaje,ColaDespegue,numPistas
  global LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas

  variableLambda = lambdaRespectoT(t)
  X = GenerarExponencial(variableLambda);

  if X>T: #El tiempo en que llega es mayor que el tiempo de simulación
    tMedColaAterrizaje = 0
    tMedColaDespegue = 0
    tMaxColaAterrizaje = 0
    tMaxColaDespegue = 0
    nMedAviones = 0
    porcentajeTPistasOcupadas = 0
  else:
    llegada_avion(X,0)
    while (t<=T and (len(LISTATLL1)>0 or len(LISTATS1)>0 or len(LISTATS2)>0 or len(LISTATS3)>0 or len(LISTATS4)>0)):
      eventTmin = 10000000
      eventplaneId = 10000000
      eventType = 10000000

      for plane in LISTATLL1:
        if(plane[0]<eventTmin):
          eventType = "Llegada"
          eventTmin = plane[0]
          eventplaneId = plane[1]

      for plane in LISTATS1:
        if(plane[0]<eventTmin):
          eventType = "Aterrizaje"
          eventTmin = plane[0]
          eventplaneId = plane[1]

      for plane in LISTATS2:
        if(plane[0]<eventTmin):
          eventType = "Guiado"
          eventTmin = plane[0]
          eventplaneId = plane[1]

      for plane in LISTATS3:
        if(plane[0]<eventTmin):
          eventType = "Embarcado"
          eventTmin = plane[0]
          eventplaneId = plane[1]

      for plane in LISTATS4:
        if(plane[0]<eventTmin):
          eventType = "Despegado"
          eventTmin = plane[0]
          eventplaneId = plane[1]

      #print(eventType,":",eventplaneId)

      if eventType == "Llegada":
        llegada_avion(eventTmin,eventplaneId)
        LISTATLL1.pop(LISTATLL1.index([eventTmin,eventplaneId]))
      elif eventType == "Aterrizaje":
        rutina_nodo_pista_aterrizaje(eventTmin,eventplaneId)
        LISTATS1.pop(LISTATS1.index([eventTmin,eventplaneId]))
      elif eventType == "Guiado":
        rutina_nodo_vehiculo(eventTmin,eventplaneId)
        LISTATS2.pop(LISTATS2.index([eventTmin,eventplaneId]))
      elif eventType == "Embarcado":
        rutina_nodo_terminal(eventTmin,eventplaneId)
        LISTATS3.pop(LISTATS3.index([eventTmin,eventplaneId]))
      elif eventType == "Despegado":
        rutina_nodo_pista_despegue(eventTmin,eventplaneId)
        LISTATS4.pop(LISTATS4.index([eventTmin,eventplaneId]))

    #Calcular tiempo medio Aterrizaje
    #Calcular tiempo maximo Aterrizaje
    tMedColAterrizaje = 0
    tMaxColaAterrizaje = 0
    numPlanes = 0
    for cola in ColaAterrizaje:
      if numPlanes>0:
        tMedColAterrizaje+=cola[0]
        if cola[0]>tMaxColaAterrizaje:
          tMaxColaAterrizaje=cola[0]
      numPlanes+=1
    tMedColAterrizaje = tMedColAterrizaje/(numPlanes-1)

    #Calcular tiempo medio Despegue
    #Calcular tiempo maximo Despegue
    tMedColaDespegue = 0
    tMaxColaDespegue = 0
    numPlanes = 0
    for cola in ColaDespegue:
      if numPlanes>0:
        tMedColaDespegue+=cola[0]
        if cola[0]>tMaxColaDespegue:
          tMaxColaDespegue=cola[0]
      numPlanes+=1
    tMedColaDespegue = tMedColaDespegue/(numPlanes-1)

    #Calcular tiempo maximo Despegue
    nMedAviones = (nmed_n1/t)+(nmed_n2/t)+(nmed_n3/t)+(nmed_n4/t)

    #Calcular porcentaje de tiempo que las pistas estan ocupadas
    porcentajeTPistasOcupadas = nmed_pistasOcupadas/(t*numPistas)

    ###############################
    ###############################
    ##### PRINTS SIMULACION #######
    ###############################
    ###############################

    avionesAterrizados = NLL1-1
    avionesDespegados = NS4
    porcentajeDespegues = NS4*100/(NLL1-1)
    porcentajeTPistasOcupadas = porcentajeTPistasOcupadas * 100

    #print("Tiempo transcurrido(min):", "{:.2f}".format(t))
    #print("Tiempo transcurrido(dias):", "{:.2f}".format(t/(60*24)))
    print("Num aviones aterrizados:", avionesAterrizados)
    print("Num aviones despegados:", avionesDespegados)
    print("Porcentaje de despegues:", "{:.2f}".format(porcentajeDespegues),"%")
    print("tMedColAterrizaje:","{:.2f}".format(tMedColAterrizaje))
    print("tMaxColaAterrizaje:","{:.2f}".format(tMaxColaAterrizaje))
    print("tMedColaDespegue:","{:.2f}".format(tMedColaDespegue))
    print("tMaxColaDespegue:","{:.2f}".format(tMaxColaDespegue))
    print("nMedAviones:","{:.2f}".format(nMedAviones))
    print("porcentajeTPistasOcupadas:","{:.2f}".format(porcentajeTPistasOcupadas),"%")

    return avionesAterrizados, avionesDespegados, porcentajeDespegues, tMedColAterrizaje, tMaxColaAterrizaje, tMedColaDespegue, tMaxColaDespegue, nMedAviones, porcentajeTPistasOcupadas

#Inicializar Listas Avion

In [ ]:
def inicializarAvion():
  global LL1,S1,LL2,S2,LL3,S3,LL4,S4,TAterrizaje,TDespegue
  LL1.append(0)
  S1.append(0)
  TAterrizaje.append(0)
  LL2.append(0)
  S2.append(0)
  LL3.append(0)
  S3.append(0)
  LL4.append(0)
  S4.append(0)
  TDespegue.append(0)


# Rutina de llegada de aviones (Aquí se realiza el aterrizaje)

In [ ]:
def llegada_avion(tsuc, idAvion):

  global n1,n2,n3,n4,TAterrizaje,TDespegue,S1,S2,S3,S4,LL1,LL2,LL3,LL4,NLL1,NS1,NS2,NS3,NS4,LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas
  global EnColaAterrizaje, ColaAterrizaje, EnColaVehiculos, ColaVehiculos, EnColaPuertas, ColaPuertas, EnColaDespegue, ColaDespegue
  global mediaAterrizajes,desviacionTipicaAterrizajes,TasaDistribucionVehiculos,TasaDistribucionPuertas,minDespegues,maxDespegues
  global t, T, numPuertas, numPistas, numVehiculos, pistasOcupadas, vehiculosOcupados, puertasOcupadas

  nmed_n1 += n1 * (tsuc-t)
  nmed_n2 += n2 * (tsuc-t)
  nmed_n3 += n3 * (tsuc-t)
  nmed_n4 += n4 * (tsuc-t)

  nmed_pistasOcupadas += pistasOcupadas * (tsuc-t)

  n1 += 1  #Sumo 1 al nº de aviones en el nodo.
  NLL1 +=1 #ID del avión
  inicializarAvion()

  LL1[idAvion] = tsuc  #añado el instante al que llega el avión con ID=NLL1 a la lista de llegadas (LL1)
  t=tsuc #Actualizo el tiempo de simulación al instante actual

  #print("Avion:" , idAvion,", Llegado: ",(t))

  variableLambda = lambdaRespectoT(t)
  Y = GenerarExponencial(variableLambda) #Genero el tiempo que tardará el siguiente avión en llegar.

  if pistasOcupadas < numPistas:
    #print("pistasOcupadas:",pistasOcupadas," Entra")
    Z = GenerarNormal(mediaAterrizajes, desviacionTipicaAterrizajes) #Genero tiempo de servicio nodo 1 (aterrizaje)
    LISTATS1.append([(t+Z),idAvion]) #Actualizo el instante en el que termina el servicio del nodo 1.
    ColaAterrizaje.append([0,idAvion])
    pistasOcupadas+=1
    #print("Avion:" , idAvion,", Aterrizaje: Previsto(",(t+Z),")")
  else:
    #print("pistasOcupadas:",pistasOcupadas," No entra")
    #print("Avion:" , idAvion,", Aterrizaje: Cola")
    EnColaAterrizaje.append(idAvion)

  if (t+Y)<T:
    #Actualizo en la lista de eventos el instante en el que llegará el siguiente avión.
    LISTATLL1.append([(t+Y),(NLL1)])
    #print("El siguiente avion " , NLL1," llegará en el instante ",(t+Y) )
  else:
    print("Horizonte T(", T, ") alcanzado, no llegaran mas aviones")


# Rutina de aterrizajes (Salida del nodo pista y entrada del nodo vehículos)

In [ ]:
def rutina_nodo_pista_aterrizaje(tsuc,idAvion):

  global n1,n2,n3,n4,TAterrizaje,TDespegue,S1,S2,S3,S4,LL1,LL2,LL3,LL4,NLL1,NS1,NS2,NS3,NS4,LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas
  global EnColaAterrizaje, ColaAterrizaje, EnColaVehiculos, ColaVehiculos, EnColaPuertas, ColaPuertas, EnColaDespegue, ColaDespegue
  global mediaAterrizajes,desviacionTipicaAterrizajes,TasaDistribucionVehiculos,TasaDistribucionPuertas,minDespegues,maxDespegues
  global t, T, numPuertas, numPistas, numVehiculos, pistasOcupadas, vehiculosOcupados, puertasOcupadas

  nmed_n1 += n1 * (tsuc-t)
  nmed_n2 += n2 * (tsuc-t)
  nmed_n3 += n3 * (tsuc-t)
  nmed_n4 += n4 * (tsuc-t)

  nmed_pistasOcupadas += pistasOcupadas * (tsuc-t)

  t=tsuc #Actualizamos tº de simulación
  pistasOcupadas-=1
  n1 -=1 #Decrementamos el nº de aviones del nodo 1 porque el avión sale del nodo 1.
  NS1 +=1 #Incrementamos el nº de salidas del nodo 1
  S1[idAvion] = tsuc #Registramos el instante en el que NS1 sale del nodo 1.

  TAterrizaje[idAvion] = S1[idAvion]-LL1[idAvion]

  #print("Avion:" , idAvion,", Aterrizaje: Realizado(",(t),")")

  n2 += 1 #Incrementamos el nº de aviones en el nodo 2
  #LL2[idAvion] = tsuc #Registramos el instante en el que el avón NLL2 llega al nodo 2. (Aquí podríamos meter directamente el ID del avión. Lo vemos)
  
  if vehiculosOcupados < numVehiculos :
    Z = GenerarExponencial(TasaDistribucionVehiculos) #Genero el tiempo que tarda en llevar el coche al terminal
    LISTATS2.append([(t+Z),(idAvion)]) #Actualizo en la lista el instante en el que saldrá del nodo 2.
    ColaVehiculos.append([0,idAvion])
    vehiculosOcupados+=1
    #print("El avion " , idAvion," terminara su guiado en el instante ",(t+Z))
  else:
    EnColaVehiculos.append(idAvion)
    #print("El avion " , idAvion," se queda en cola de guiado")


  if len(EnColaAterrizaje)>0:
    W = GenerarNormal(mediaAterrizajes, desviacionTipicaAterrizajes) #Genero el tiempo que tardará en aterrizar el avión de la cola
    LISTATS1.append([(t+W),EnColaAterrizaje[0]])  #Actualizo el instante en el que saldrá el avión del nodo 1.
    ColaAterrizaje.append([t-LL1[EnColaAterrizaje[0]],EnColaAterrizaje[0]])
    #print("Avion:" , EnColaAterrizaje[0],", Aterrizaje: Previsto(",(t+W),")")
    pistasOcupadas+=1
    EnColaAterrizaje.pop(0)
  elif len(EnColaDespegue)>0:
    Y = GenerarUniforme(minDespegues,maxDespegues) #Genero el tiempo que tarda el avión en realizar el despegue
    LISTATS4.append([(t+Y),(EnColaDespegue[0])]) #Actualizo en la lista el instante en el que saldrá del nodo 2.
    ColaDespegue.append([t-S3[EnColaDespegue[0]],EnColaDespegue[0]])
    #print("El siguiente avion " , EnColaDespegue[0] ," en despegara en el instante ",(t+Y) )
    pistasOcupadas+=1
    EnColaDespegue.pop(0)


# Rutina de vehículos (Salida del nodo vehículo y entrada del nodo terminal)

In [ ]:
def rutina_nodo_vehiculo(tsuc, idAvion):

  global n1,n2,n3,n4,TAterrizaje,TDespegue,S1,S2,S3,S4,LL1,LL2,LL3,LL4,NLL1,NS1,NS2,NS3,NS4,LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas
  global EnColaAterrizaje, ColaAterrizaje, EnColaVehiculos, ColaVehiculos, EnColaPuertas, ColaPuertas, EnColaDespegue, ColaDespegue
  global mediaAterrizajes,desviacionTipicaAterrizajes,TasaDistribucionVehiculos,TasaDistribucionPuertas,minDespegues,maxDespegues
  global t, T, numPuertas, numPistas, numVehiculos, pistasOcupadas, vehiculosOcupados, puertasOcupadas

  nmed_n1 += n1 * (tsuc-t)
  nmed_n2 += n2 * (tsuc-t)
  nmed_n3 += n3 * (tsuc-t)
  nmed_n4 += n4 * (tsuc-t)

  nmed_pistasOcupadas += pistasOcupadas * (tsuc-t)

  t=tsuc
  vehiculosOcupados-=1
  n2 -= 1 #Decrementamos que el avión ha salido del nodo 2
  NS2 += 1 #Aumentamos en 1 el número de salidas del nodo 2
  S2[idAvion] = t #Registramos el instante en el que el avión NS2 sale del nodo 2

  n3 += 1
  #LL3[idAvion] = t

  if vehiculosOcupados < numVehiculos and len(EnColaVehiculos)>0:
    Y = GenerarExponencial(TasaDistribucionVehiculos) #Genero el tiempo que tarda en llevar el coche al terminal
    LISTATS2.append([(t+Y),(idAvion)]) #Actualizo en la lista el instante en el que saldrá del nodo 2.
    ColaVehiculos.append([t-S1[EnColaVehiculos[0]],EnColaVehiculos[0]])
    EnColaVehiculos.pop(0)
    vehiculosOcupados+=1

  if puertasOcupadas < numPuertas :
    Z = GenerarExponencial(TasaDistribucionPuertas) #Genero el tiempo que tarda en preparar desembarque y embarque
    LISTATS3.append([(t+Z),(idAvion)]) #Actualizo en la lista el instante en el que saldrá del nodo 3.
    ColaPuertas.append([0,idAvion])
    puertasOcupadas+=1
  else:
    EnColaPuertas.append(idAvion)



# Rutina de terminal (Salida del nodo terminal y entrada en nodo pista para despegar)

In [ ]:
def rutina_nodo_terminal(tsuc, idAvion):

  global n1,n2,n3,n4,TAterrizaje,TDespegue,S1,S2,S3,S4,LL1,LL2,LL3,LL4,NLL1,NS1,NS2,NS3,NS4,LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas
  global EnColaAterrizaje, ColaAterrizaje, EnColaVehiculos, ColaVehiculos, EnColaPuertas, ColaPuertas, EnColaDespegue, ColaDespegue
  global mediaAterrizajes,desviacionTipicaAterrizajes,TasaDistribucionVehiculos,TasaDistribucionPuertas,minDespegues,maxDespegues
  global t, T, numPuertas, numPistas, numVehiculos, pistasOcupadas, vehiculosOcupados, puertasOcupadas

  nmed_n1 += n1 * (tsuc-t)
  nmed_n2 += n2 * (tsuc-t)
  nmed_n3 += n3 * (tsuc-t)
  nmed_n4 += n4 * (tsuc-t)

  nmed_pistasOcupadas += pistasOcupadas * (tsuc-t)

  t=tsuc
  puertasOcupadas-=1
  n3 -= 1 #Decrementamos que el avión ha salido del nodo 2
  NS3 += 1 #Aumentamos en 1 el número de salidas del nodo 2
  S3[idAvion] = t #Registramos el instante en el que el avión NS2 sale del nodo 2

  n4 += 1
  #LL4[idAvion] = t

  if len(EnColaPuertas)>0:
    Y = GenerarExponencial(TasaDistribucionTerminal) #Genero el tiempo que tarda en llevar el coche al terminal
    LISTATS3.append([(t+Y),(idAvion)]) #Actualizo en la lista el instante en el que saldrá del nodo 2.
    ColaPuertas.append([t-S2[EnColaPuertas[0]],EnColaPuertas[0]])
    EnColaPuertas.pop(0)
    puertasOcupadas+=1

  if pistasOcupadas<numPistas:
    Z = GenerarUniforme(minDespegues,maxDespegues) #Genero el tiempo que tarda el avión en realizar el despegue
    LISTATS4.append([(t+Z),(idAvion)]) #Actualizo en la lista el instante en el que saldrá del nodo 3.
    pistasOcupadas+=1
    ColaDespegue.append([0,idAvion])
  else:
    EnColaDespegue.append(idAvion)


# Rutina despegue (Salida del nodo pista para despegar y dejar el aeropuerto)

In [ ]:
def rutina_nodo_pista_despegue(tsuc, idAvion):

  global n1,n2,n3,n4,TAterrizaje,TDespegue,S1,S2,S3,S4,LL1,LL2,LL3,LL4,NLL1,NS1,NS2,NS3,NS4,LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas
  global EnColaAterrizaje, ColaAterrizaje, EnColaVehiculos, ColaVehiculos, EnColaPuertas, ColaPuertas, EnColaDespegue, ColaDespegue
  global mediaAterrizajes,desviacionTipicaAterrizajes,TasaDistribucionVehiculos,TasaDistribucionPuertas,minDespegues,maxDespegues
  global t, T, numPuertas, numPistas, numVehiculos, pistasOcupadas, vehiculosOcupados, puertasOcupadas

  nmed_n1 += n1 * (tsuc-t)
  nmed_n2 += n2 * (tsuc-t)
  nmed_n3 += n3 * (tsuc-t)
  nmed_n4 += n4 * (tsuc-t)
  
  nmed_pistasOcupadas += pistasOcupadas * (tsuc-t)

  t=tsuc
  pistasOcupadas-=1
  n4 -= 1 #Decrementamos que el avión ha salido del nodo 1 en la parte de despegues
  NS4 += 1 #Aumentamos en 1 el número de salidas del nodo 1 en la parte de despegues
  S4[idAvion] = t #Registramos el instante en el que el avión NS2 sale del nodo 2

  TDespegue[idAvion] = S4[idAvion]-S3[idAvion]

  if len(EnColaAterrizaje)>0:
    W = GenerarNormal(mediaAterrizajes, desviacionTipicaAterrizajes) #Genero el tiempo que tardará en aterrizar el avión de la cola
    LISTATS1.append([(t+W),EnColaAterrizaje[0]])  #Actualizo el instante en el que saldrá el avión del nodo 1.
    ColaAterrizaje.append([t-LL1[EnColaAterrizaje[0]],EnColaAterrizaje[0]])
    #print("Avion:" , EnColaAterrizaje[0],", Aterrizaje: Previsto(",(t+W),")")
    pistasOcupadas+=1   
    EnColaAterrizaje.pop(0)
  elif len(EnColaDespegue)>0:
    Y = GenerarUniforme(minDespegues,maxDespegues) #Genero el tiempo que tarda el avión en realizar el despegue
    LISTATS4.append([(t+Y),(EnColaDespegue[0])]) #Actualizo en la lista el instante en el que saldrá del nodo 2.
    ColaDespegue.append([t-S3[EnColaDespegue[0]],EnColaDespegue[0]])
    pistasOcupadas+=1
    EnColaDespegue.pop(0)

#Funcion generar Poisson

In [ ]:
def lambdaRespectoT(t):
  tiempo = t/60
  tiempo = tiempo%24
  if(tiempo<5):
    lambdaInT = (2*tiempo/5)+5
  elif(tiempo<8):
    lambdaInT = (-1*tiempo/3)+(26/3)
  elif(tiempo<15):
    lambdaInT = (3*tiempo/7)+(18/7)
  elif(tiempo<17):
    lambdaInT = (-3*tiempo/2)+(63/2)
  elif(tiempo<=24):
    lambdaInT = (-1*tiempo/7)+(59/7)
  return lambdaInT

def GenerarExponencial(variableLambda):
  '''u = np.random.rand()
  exponencial = (-1/variableLambda)*np.log(u)'''

  exponencial = np.random.exponential(variableLambda)
  return exponencial

def GenerarNormal(media, desviacionTipica):
  normal = np.random.normal(media, desviacionTipica)
  return normal

def GenerarUniforme(min, max):
  uniforme = random.uniform(min, max)
  return uniforme

print("tAleatorioLlegada:",GenerarExponencial(5)) #Exponencial Tasa lambda llegada ejemplo t=0 --> lambda=5
print("tAleatorioAterrizaje:",GenerarNormal(mediaAterrizajes,desviacionTipicaAterrizajes)) #Normal de aterrizajes
print("tAleatorioVehiculo:",GenerarExponencial(TasaDistribucionVehiculos)) #Exponencial Vehiculos
print("tAleatorioTerminal:",GenerarExponencial(TasaDistribucionPuertas)) #Exponencial Puertas o Terminal
print("tAleatorioDespegue:",GenerarUniforme(minDespegues, maxDespegues)) #Uniforme Despegues

tAleatorioLlegada: 1.964412292200586
tAleatorioAterrizaje: 12.559076626770324
tAleatorioVehiculo: 2.2703218763656685
tAleatorioTerminal: 10.901547059723157
tAleatorioDespegue: 12.361889553123877


#Reiniciar variables

In [ ]:
t=0

NLL1 = 0
NS1 = 0
NS2 = 0
NS3 = 0
NS4 = 0
n1 = 0
n2 = 0
n3 = 0
n4 = 0
nmed_n1 = 0
nmed_n2 = 0
nmed_n3 = 0
nmed_n4 = 0
nmed_pistasOcupadas = 0

pistasOcupadas=0
vehiculosOcupados=0
puertasOcupadas=0

EnColaAterrizaje = []
EnColaVehiculos = []
EnColaPuertas = []
EnColaDespegue = []

LL1 = [0] #Instante de llegada de un avion nodo 1
LL2 = [0] #Instante de llegada de un avion nodo 2
LL3 = [0] #Instante de llegada de un avion nodo 3
LL4 = [0] #Instante de llegada de un avion nodo 1 en la parte de despegues

ColaAterrizaje = [0] #Tiempo en cola Aterrizaje
ColaVehiculos = [0] #Tiempo en cola Vehiculos
ColaPuertas = [0] #Tiempo en cola Aterrizaje
ColaDespegue = [0] #Tiempo en cola Vehiculos

S1 = [0] #Instante de llegada de un avion
S2 = [0] #Instante de llegada de un avion
S3 = [0] #Instante de llegada de un avion
S4 = [0] #Instante de llegada de un avion

TAterrizaje = [0]
TDespegue = [0]

LISTATLL1 = []
LISTATS1 = []
LISTATS2 = []
LISTATS3 = []
LISTATS4 = []

def reiniciarVariables():
  global n1,n2,n3,n4,TAterrizaje,TDespegue,S1,S2,S3,S4,LL1,LL2,LL3,LL4,NLL1,NS1,NS2,NS3,NS4,LISTATLL1,LISTATS1,LISTATS2,LISTATS3,LISTATS4,nmed_n1,nmed_n2,nmed_n3,nmed_n4,nmed_pistasOcupadas
  global EnColaAterrizaje, ColaAterrizaje, EnColaVehiculos, ColaVehiculos, EnColaPuertas, ColaPuertas, EnColaDespegue, ColaDespegue
  global mediaAterrizajes,desviacionTipicaAterrizajes,TasaDistribucionVehiculos,TasaDistribucionPuertas,minDespegues,maxDespegues
  global t, T, numPuertas, numPistas, numVehiculos, pistasOcupadas, vehiculosOcupados, puertasOcupadas

  t=0

  NLL1 = 0
  NS1 = 0
  NS2 = 0
  NS3 = 0
  NS4 = 0
  n1 = 0
  n2 = 0
  n3 = 0
  n4 = 0
  nmed_n1 = 0
  nmed_n2 = 0
  nmed_n3 = 0
  nmed_n4 = 0
  nmed_pistasOcupadas = 0

  pistasOcupadas=0
  vehiculosOcupados=0
  puertasOcupadas=0

  EnColaAterrizaje = []
  EnColaVehiculos = []
  EnColaPuertas = []
  EnColaDespegue = []

  LL1 = [0] #Instante de llegada de un avion nodo 1
  LL2 = [0] #Instante de llegada de un avion nodo 2
  LL3 = [0] #Instante de llegada de un avion nodo 3
  LL4 = [0] #Instante de llegada de un avion nodo 1 en la parte de despegues

  ColaAterrizaje = [0] #Tiempo en cola Aterrizaje
  ColaVehiculos = [0] #Tiempo en cola Vehiculos
  ColaPuertas = [0] #Tiempo en cola Aterrizaje
  ColaDespegue = [0] #Tiempo en cola Vehiculos

  S1 = [0] #Instante de llegada de un avion
  S2 = [0] #Instante de llegada de un avion
  S3 = [0] #Instante de llegada de un avion
  S4 = [0] #Instante de llegada de un avion

  TAterrizaje = [0]
  TDespegue = [0]

  LISTATLL1 = []
  LISTATS1 = []
  LISTATS2 = []
  LISTATS3 = []
  LISTATS4 = []

#Lanzar Simulacion con parametros deseados

In [ ]:
###############################
###############################
##### VARIABLES INICIALES #####
###############################
###############################

# Tiempo de simulacion en minutos(1 mes = 30 dias = 720 horas = 43200 minutos)
T=30*24*60

mediaAterrizajes = 10.0076
desviacionTipicaAterrizajes = 3.0377
TasaDistribucionVehiculos = 2
TasaDistribucionPuertas = 45.9012
minDespegues = 10
maxDespegues = 15

###############################
###############################
##### LANZAR SIMULACION #######
###############################
###############################

print("-------------------")
print("Simulacion Original")
print("-------------------")

reiniciarVariables()
# Numero de pistas
numPistas = 3
# Numero de vehiculos
numVehiculos = 20
#Numero de puertas en la terminal
numPuertas = 50

simul_main()

print("-------------------")
print("Simulacion 4 Pistas")
print("-------------------")
reiniciarVariables()
# Numero de pistas
numPistas = 4
# Numero de vehiculos
numVehiculos = 20
#Numero de puertas en la terminal
numPuertas = 50

simul_main()

print("-------------------")
print("Simulacion 25 Vehiculos")
print("-------------------")
reiniciarVariables()
# Numero de pistas
numPistas = 3
# Numero de vehiculos
numVehiculos = 25
#Numero de puertas en la terminal
numPuertas = 50

a=simul_main()
#FIN

-------------------
Simulacion Original
-------------------
Horizonte T( 43200 ) alcanzado, no llegaran mas aviones
Num aviones aterrizados: 6904
Num aviones despegados: 4817
Porcentaje de despegues: 69.77 %
tMedColAterrizaje: 5.55
tMaxColaAterrizaje: 45.66
tMedColaDespegue: 6384.86
tMaxColaDespegue: 12801.93
nMedAviones: 1031.02
porcentajeTPistasOcupadas: 99.86 %
-------------------
Simulacion 4 Pistas
-------------------
Horizonte T( 43200 ) alcanzado, no llegaran mas aviones
Num aviones aterrizados: 6891
Num aviones despegados: 6878
Porcentaje de despegues: 99.81 %
tMedColAterrizaje: 2.80
tMaxColaAterrizaje: 22.55
tMedColaDespegue: 46.14
tMaxColaDespegue: 246.24
nMedAviones: 18.89
porcentajeTPistasOcupadas: 89.88 %
-------------------
Simulacion 25 Vehiculos
-------------------
Horizonte T( 43200 ) alcanzado, no llegaran mas aviones
Num aviones aterrizados: 6827
Num aviones despegados: 4903
Porcentaje de despegues: 71.82 %
tMedColAterrizaje: 5.20
tMaxColaAterrizaje: 29.40
tMedColaDe